## Imports

In [1]:
import googlemaps
import pandas as pd
import numpy as np
import re
import os
import folium

## Read-in

In [3]:
df = pd.read_csv("ChicagoPortfolioMap_04_26_24 - Sheet1.csv")

## Google Maps Geolocater

In [6]:
%store -r google_maps_API_Key
gmaps_key = googlemaps.Client(key=google_maps_API_Key)

In [7]:
df['full_address'] = df['full_address'] + ' Chicago'

In [8]:
def geocode(add):
    g = gmaps_key.geocode(add)
    lat = g[0]["geometry"]["location"]["lat"]
    lng = g[0]["geometry"]["location"]["lng"]
    return (lat, lng)

df['geocoded'] = df['full_address'].apply(geocode)

df['geocoded'] = df['geocoded'].astype(str)
df[['lat', 'lon']] = df['geocoded'].str.strip('()').str.split(', ', expand=True)
df['lat'] = df['lat'].astype(float)
df['lon'] = df['lon'].astype(float)

In [9]:
pd.set_option('display.max_columns',None)

In [10]:
df.columns

Index(['full_address', 'portfolio_name', 'color', 'geocoded', 'lat', 'lon'], dtype='object')

In [21]:
def popup_html(row):
    portfolio_name = row['portfolio_name']
    address = row['full_address']
    
    html = '''<!DOCTYPE html>
    <html>
    <strong>Portfolio: </strong>{}'''.format(portfolio_name) + '''<br>
    <strong>Address: </strong>{}'''.format(address) + '''<br>
    </html>
    '''
    return html

In [22]:
df['color'].value_counts()

color
red       9
purple    6
orange    3
blue      1
Name: count, dtype: int64

In [23]:
from folium.plugins import MarkerCluster

title_html = '''
              <h3 align="center" style="font-size:16px"><b>{}</b></h3>
             '''.format(f"F Street's $35M South Side Acquisitions")

### Create map container ###
m = folium.Map(location=df[["lat", "lon"]].mean().to_list(),zoom_start=11.5,tiles='OpenStreetmap')


for index, row in df.iterrows():
    marker = folium.Marker(
        location=[row['lat'], row['lon']],
        radius=5,
        fill=True,
        icon=folium.Icon(color=row['color']),
        popup=folium.Popup(popup_html(row), max_width=400))
    marker.add_to(m)
            
# Display map
m

In [24]:
m.save('index.html')

## Map URL snagger

Map template URL: `https://trd-digital.github.io/trd-news-interactive-maps/{map-folder-name}`

In [20]:
base_name = 'https://trd-digital.github.io/trd-news-interactive-maps/'

cwd = os.getcwd()

cwd = cwd.split('/')

final_name = base_name + cwd[-1]
print(final_name)

https://trd-digital.github.io/trd-news-interactive-maps/CookCountyPortfolioMap_04_26_24


In [30]:
map_iframe = f'''<figure>
 <div class="container">
   <div class="iframe-wrap">
   <iframe src="{final_name}" width="100%" height="600" frameBorder="0" scrolling="no"></iframe>
  </div>
</div>
  <figcaption align="right"><a href="https://leafletjs.com/">Leaflet</a> map created by Adam Farence | Data by © <a href="https://www.openstreetmap.org/#map=4/38.01/-95.84"> OpenStreetMap</a>, under <a href="https://www.openstreetmap.org/copyright">ODbl.</a></figcaption>
</figure>'''

print(map_iframe)

<figure>
 <div class="container">
   <div class="iframe-wrap">
   <iframe src="https://trd-digital.github.io/trd-news-interactive-maps/CookCountyPortfolioMap_04_26_24" width="100%" height="600" frameBorder="0" scrolling="no"></iframe>
  </div>
</div>
  <figcaption align="right"><a href="https://leafletjs.com/">Leaflet</a> map created by Adam Farence | Data by © <a href="https://www.openstreetmap.org/#map=4/38.01/-95.84"> OpenStreetMap</a>, under <a href="https://www.openstreetmap.org/copyright">ODbl.</a></figcaption>
</figure>
